To evaluate simplification quality 2 baselines will be used:

* Simple trunctuation baseline: leave only a certain number of the first words of a sentence
* GPT generation (finetuned russian gpt-3 model from Sber)

As the test set I will use the dev part of the Russian dataset collected via Toloka

# Trunctuation Baseline

##Loading data...

In [ ]:
! pip install textstat
! pip install --upgrade language_tool_python

In [1]:
import nltk
import textstat
import language_tool_python
nltk.download('punkt')
import re
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# test_data2 = pd.read_csv('/content/drive/MyDrive/SImplification_models/hidden_test_sents.csv')
# test_data = pd.read_csv('/content/drive/MyDrive/SImplification_models/public_test_sents.csv')#, error_bad_lines=False, engine='python')
# full_test = pd.concat((test_data, test_data2))
# test_data.head()

In [ ]:
! wget https://raw.githubusercontent.com/dialogue-evaluation/RuSimpleSentEval/main/dev_sents.csv
test_data = pd.read_csv('/content/dev_sents.csv')
test_data.shape[0]

First look at the test data (which is actually a dev part from the contest)

In [ ]:
test_data['trunctuation_bs'] = test_data['INPUT:source'].apply(lambda x: ' '.join(x.split()[:int(len(x.split())*0.6)])+'.')
test_data.sample(5)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs
225,686,"В Китае передача идёт в основном в кругу семьи, внутрибольничная передача в данной стране для инфекции не характерна.","В Китае обычно заражение происходит от родственников, а не из-за посещения больницы.","В Китае передача идёт в основном в кругу семьи, внутрибольничная."
1073,3229,"За столетия потрясений численность этнических вавилонян в Южной Месопотамии сократилась, большинство населения там составляли халдеи.",За последние несколько столетий сократилось население вавилонян в Южной Месопотамии,За столетия потрясений численность этнических вавилонян в Южной Месопотамии.
1391,4178,"Красный Крест сообщил, что не может начать работу в Карабахе из-за нарушений перемирия.",Работа красного креста не возможра в аарабахе по причине нарушения перемирия.,"Красный Крест сообщил, что не может начать."
3146,9234,"Туристическая индустрия Ирана серьёзно пострадала в результате ирано-иракской войны, однако в настоящее время возрождается.","Туриндустрия Ирана пострадала в результате ирано-иракской войны, но сейчас возрождается.",Туристическая индустрия Ирана серьёзно пострадала в результате ирано-иракской.
844,2478,"Гениальнейшим поэтом Польши и одновременно одним из великих мировых поэтов является Адам Мицкевич, признанный вождь польского романтизма.",Адам Мицкевич - всемирно известный польский поэт.\r\n,Гениальнейшим поэтом Польши и одновременно одним из великих мировых поэтов.


# Second baseline will be automatically generated by GPT-3

I follow the procedure described in Sber repo
# Finetune RuGPTs in megatron without deepspeed

In [ ]:
!pip3 install transformers==3.5.0

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh
!git clone  https://github.com/sberbank-ai/ru-gpts

File: /usr/local/lib/python3.7/dist-packages/transformers/trainer_pt_utils.py

> `if version.parse(torch.__version__) <= version.parse("1.4.1") or version.parse(torch.__version__) > version.parse("1.7.0"):`

## Download files

Here we already need the translated data to tune a model

In [ ]:
! gdown https://drive.google.com/uc?id=1dB3X-Wx8qU_5nDG_pxAmLvo5H_sgnHrE
! gdown https://drive.google.com/uc?id=1bJo8TagTGKa0uyppQRqsHrKHyYO5tcZc
! gdown https://drive.google.com/uc?id=11lqipq6ggrgCk8bVxQ4-uuPVMCKN5ebU

import pandas as pd
df_dev_google = pd.read_csv('/content/wiki_dev_cleaned_translated_sd.csv')
df_test_google = pd.read_csv('/content/wiki_test_cleaned_translated_sd.csv')
df_train_google = pd.read_csv('/content/wiki_train_cleaned_translated_sd.csv')

## Prepare data for parallel
We use custom implementation of distributed dataset. For training and evaluating we should specify file `file.list` with list of paths to txt files. All files from `file.list` will be splitted between aviable GPUs. The logic of splitting is described by the following code:

```python
shard_size = len(files) // world_size
shard_start = rank * shard_size
shard_end = (rank + 1) * shard_size
files = files[shard_start:shard_end]
```

For more details please see full code of dataset: `src.dataset_rugpt3.RuGpt3TextDataset`.

In [ ]:
with open('train.txt', 'w+') as f:
  for i,j in list(zip(df_train_google.target_x.values, df_train_google.target_y.values)):
    f.write('<s>'+i+'\n'+j+'\n') # +'<\s>'

with open('valid.txt', 'w+') as f:
  for i,j in list(zip(df_dev_google.target_x.values, df_dev_google.target_y.values)):
    f.write('<s>'+i+'\n'+j+'\n') # +'<\s>'

!echo /content/train.txt > train.list
!echo /content/valid.txt > valid.list

## Train
Load model from Huggingface and finetune on essays.

This will take arount ten minutes.

In [ ]:
! rm -r /content/model
! rm -r /content/model_hf

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/pretrain_gpt3.py \
  --train-data-path "/content/train.list" \
  --test-data-path "/content/valid.list" \
  --max-files-per-process 100 \
  --logging-dir="log" \
  --save model \
  --load-huggingface sberbank-ai/rugpt3small_based_on_gpt2 \
  --save-interval 1000 \
  --log-interval 100 \
  --eval-interval 1000 \
  --eval-iters 100 \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --batch-size 1 \
  --seq-length 512 \
  --max-position-embeddings 2048 \
  --train-iters 2000 \
  --resume-dataloader \
  --distributed-backend "nccl" \
  --lr 0.00015 \
  --lr-decay-style "cosine" \
  --lr-decay-iters 3200 \
  --clip-grad 0.5 \
  --warmup .004


2021-04-13 16:44:11.667377: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
> initializing model parallel with size 1
Pretrain GPT3 model
arguments:
  attention_dropout ............ 0.1
  num_attention_heads .......... 12
  hidden_size .................. 768
  intermediate_size ............ None
  num_layers ................... 12
  layernorm_epsilon ............ 1e-05
  hidden_dropout ............... 0.1
  max_position_embeddings ...... 2048
  vocab_size ................... 30522
  deep_init .................... False
  make_vocab_size_divisible_by . 8
  cpu_optimizer ................ False
  cpu_torch_adam ............... False
  sparse_mode .................. all
  fp16 ......................... False
  fp32_embedding ............... False
  fp32_layernorm ............... False
  fp32_tokentypes .............. False
  fp32_allreduce ....

At the end of training output should be something like this:

"-----------------------------------------------------------------------------------------

 validation loss at the end of training for test data | LM loss: 2.7927 | LM PPL: 16.325
 
-----------------------------------------------------------------------------------------"

In [ ]:
# ! cp -r /content/model /content/drive/MyDrive/SImplification_models

## Generate

Load pretrained model from dir and generate.

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/generate_samples.py \
  --load /content/drive/MyDrive/SImplification_models/model \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --batch-size 1 \
  --seq-length 50 \
  --max-position-embeddings 2048 \
  --distributed-backend "nccl" \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim


### Convert checkpoint to Huggingface format

In [ ]:
# /content/model/

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/convert2huggingface.py \
  --load model \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --max-position-embeddings 2048 \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim \
  --export-huggingface /content/drive/MyDrive/SImplification_models/model_hf


In [ ]:
!ls model_hf

#### Test load

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re

def create_model_and_tok(
        model_name="/content/drive/MyDrive/SImplification_models/model_hf" #/content/drive/MyDrive/SImplification_models/
):
    print('loading from {}'.format(model_name))
    gpt_model = GPT2LMHeadModel.from_pretrained(model_name)
    gpt_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    gpt_model.cuda()
    gpt_model.eval()
    gpt_tokenizer.padding_side = "left"
    gpt_tokenizer.pad_token = '[PAD]'
    gpt_tokenizer.encoder['[PAD]'] = 50256
    gpt_model.config.pad_token_id = gpt_model.config.eos_token_id
    return gpt_model, gpt_tokenizer


def batch_generator(
        list_of_sentences,
        size=16
):
    num_batch = len(list_of_sentences)//size
    for index in range(num_batch):
        yield list_of_sentences[index*size:(index+1)*size]
    yield list_of_sentences[num_batch*size:]


def get_outputs(
        model,
        tokenizer,
        list_of_sentences,
        seq_len=11, 
        batch_size=16
):
    result = []
    for batch in batch_generator(list_of_sentences, batch_size):
        max_length = len(max(list_of_sentences, key=lambda x: x.split()).split()) + 1
        encodings_dict = tokenizer.batch_encode_plus(batch, max_length=max_length, pad_to_max_length=True, add_special_tokens=False)
        input_ids = torch.tensor(encodings_dict['input_ids']).cuda()
        attn_mask = torch.tensor(encodings_dict['attention_mask']).cuda()

        outputs = model.generate(input_ids,
                                 attention_mask=attn_mask,
                                 do_sample=True,
                                 max_length=40, #1000+ max_length,
                                 top_k=10,
                                 top_p=0.95,
                                 repetition_penalty=5.0,
                                 num_return_sequences=1)
                                 # no_repeat_ngram_size=3)

        outputs = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
        outputs = [text[:re.search(r'<\\?s?>?', text).start()].strip() if  re.search(r'<\\?s?>?', text) is not None else text.strip() for text  in outputs]
        result.extend(outputs)
    return result


if __name__ == '__main__':
    prompt_text = [
        "Британская транспортная комиссия (BTC) была создана послевоенным лейбористским правительством Клемента Эттли в рамках его программы национализации для надзора за железными дорогами, каналами и автомобильными грузовыми перевозками в Великобритании (в Северной Ирландии было отдельное транспортное управление Ольстера).",
        "Отношения Фицджеральда с Гранцем еще больше укрепились, когда он стал ее менеджером, хотя прошло почти десять лет, прежде чем он смог записать ее на одном из своих многочисленных лейблов.",
        "Они вымерли на материке, а оставшиеся популяции были ограничены 32 прибрежными островами до первого выпуска с материка в сильно огороженный и находящийся под наблюдением заповедник Карори в 2005 году.",
        "Вместе с Bryozoa и Brachiopoda форониды принадлежат к лофофоратам, которые иногда рассматриваются как один тип."]

    # prompt_text = ["Python популярен среди индивидуальных разработчиков, но также используется крупными компаниями в достаточно серьёзных продуктах, ориентированных на получение прибыли.",
    #                "Официальной столицей государства, согласно конституции Нидерландов, является Амстердам, где монарх приносит присягу на верность Конституции.",
    #                "После первого неудачного сражения Джексон отступил на юг по долине, затем атаковал и разбил отряд Фримонта."]
    # /mnt/sdb1/data/aizhevskaya/ft_model_new/checkpoint-3000
    # "/mnt/sdb1/data/aizhevskaya/ft_model_new/three_epoch"
    # /mnt/sdb1/data/aizhevskaya/ft_model/checkpoint-30000
    # "/mnt/sdb1/data/aizhevskaya/ft_model_new/simpl1"
    model, tokenizer = create_model_and_tok()
    # perplexity = tensor(16.6698)
    texts = get_outputs(model, tokenizer, prompt_text)
    for text in texts:
        print(text)



loading from /content/drive/MyDrive/SImplification_models/model_hf


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_sp

Британская транспортная комиссия (BTC) была создана послевоенным лейбористским правительством Клемента Эттли в рамках его программы национализации, а затем преобразована из Британской транспортной комиссии
Отношения Фицджеральда с Гранцем еще больше укрепились, когда он стал ее менеджером, хотя прошло почти десять лет, прежде чем она перешла на работу в Microsoft.
Они вымерли на материке, а оставшиеся популяции были ограничены 32 прибрежными островами до первого выпуска с материка в сильно огороженный и густонаселенный регион Северной Америки.
Вместе с Bryozoa и Brachiopoda форониды принадлежат к лофофоратам, которые иногда рассматриваются как этологи.


# Look at the data

In [ ]:
model, tokenizer = create_model_and_tok()
texts = get_outputs(model, tokenizer, test_data['INPUT:source'], batch_size=32)
test_data['gpt_bs'] = texts
test_data.sample(3)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs
2595,7703,Рамзан Кадыров назвал возрождение многонационального сообщества республики одной из приоритетных задач нового руководства республики.,"Рамзан Кадыров считает, что главная задача руководства - возродить многонациональное общество республики.",Рамзан Кадыров назвал возрождение многонационального сообщества республики одной.,Рамзан Кадыров назвал возрождение многонационального сообщества республики одной из приоритетных задач.
3175,9354,"Улучшение оксигенации лёгких, наблюдаемое при интенсификации дыхания в разрежённом воздухе горных курортов, способствует торможению роста и размножения микобактерий.","Улучшение обогащения лёгких кислородом, которое наблюдается при усилении функций дыхания в разрежённом воздухе горных курортов, способствует также размножению и росту микобактерий.","Улучшение оксигенации лёгких, наблюдаемое при интенсификации дыхания в разрежённом воздухе.","Улучшение оксигенации лёгких, наблюдаемое при интенсификации окиснения урана (AO), является важным механизмом очистки и удаления отходов в легких."
808,2391,"Вторично одичавшие кошки часто живут уединённо и охотятся в одиночку, но иногда образуют небольшие колонии из нескольких самок с котятами.","Как правило, вторично одичавшие кошки и живут и охотятся в одиночку, но бывает, что иногда они образуют отдельные поселения, в которые входят несколько самок и их котята.","Вторично одичавшие кошки часто живут уединённо и охотятся в одиночку, но иногда.",Вторично одичавшие кошки часто живут уединённо и охотятся в дикой природе.


## SAVING BASELINES....

# Evaluation

## cosine similarity

In [ ]:
from transformers import RobertaTokenizer, RobertaModel, AutoConfig, AutoTokenizer, AutoModelForMaskedLM
device = "cuda" if torch.cuda.is_available() else "cpu"
config = AutoConfig.from_pretrained("DeepPavlov/rubert-base-cased") # "roberta-base" 'xlm-mlm-100-1280' 'xlm-roberta-base' 'bert-base-multilingual-cased'
config.output_hidden_states = True

tok = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModelForMaskedLM.from_pretrained("DeepPavlov/rubert-base-cased", config=config)
model.to(device)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# shape should be [1, something (768, ex)]

# import numpy as np
# def cs(a, b):
#   return (a @ b.T)/(np.linalg.norm(a)*np.linalg.norm(b))

def calc_cos_sim(df, model,tok, x, y, column_name):
    Cos_sim= []
    for index, row in df.iterrows():
        
        # original
          sentence_A = tok.encode(row[x], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_A = sentence_A.to(device)
          output = model(sentence_A)
          sent_emb = output[-1][0]
          emb_source = sent_emb.mean(axis=1)
          emb_source = emb_source.cpu().detach().numpy()

          sentence_B = tok.encode(row[y], padding='max_length', max_length=50, truncation=True, return_tensors='pt')
          sentence_B = sentence_B.to(device)
          output = model(sentence_B)
          sent_emb = output[-1][0]
          emb_target= sent_emb.mean(axis=1)
          emb_target = emb_target.cpu().detach().numpy()

          cos_val = cosine_similarity(emb_source.reshape(emb_source.shape[0], -1), emb_target.reshape(emb_target.shape[0], -1))[0][0]
          Cos_sim.append(cos_val)
    df[column_name] = Cos_sim

In [ ]:
calc_cos_sim(test_data, model, tok, 'INPUT:source', 'OUTPUT:output', 'cos_sim_ref')
calc_cos_sim(test_data, model, tok, 'INPUT:source', 'trunctuation_bs', 'cos_sim_tr_bs')
calc_cos_sim(test_data, model, tok, 'INPUT:source', 'gpt_bs', 'cos_sim_gpt_bs')

In [ ]:
test_data.to_csv('/content/drive/MyDrive/SImplification_models/test_baselines.csv', index=False, sep='\t')
test_data.head(3)

,Unnamed: 0,INPUT:source,OUTPUT:output,trunctuation_bs,gpt_bs,cos_sim_ref,cos_sim_tr_bs,cos_sim_gpt_bs
0,3,14 декабря 1944 года рабочий посёлок Ички был ...,14 декабря 1944 года рабочий посёлок Ички пере...,14 декабря 1944 года рабочий посёлок Ички был ...,14 декабря 1944 года рабочий посёлок Ички был ...,0.924978,0.928283,0.970611
1,4,1960 году была выпущена модель 172A. Изменени...,В 1960 году вышла модель 172А. Отличие в хвост...,1960 году была выпущена модель 172A. Изменения...,1960 году была выпущена модель 172A. Изменения...,0.962903,0.954120,0.957274
2,5,1960 году была выпущена модель 172A. Изменени...,В выпущенной в 1960 году модель имела изменени...,1960 году была выпущена модель 172A. Изменения...,1960 году была выпущена модель 172A. Изменения...,0.950816,0.954120,0.970414


## BLEU SARI (only SARI so far)

In [ ]:
# import pandas as pd
# test_data = pd.read_csv('/content/drive/MyDrive/MT_sentence_simpl/test_baselines.csv', sep='\t')

In [ ]:
! git clone https://github.com/feralvam/easse
! git clone https://github.com/Andoree/sent_simplification.git
%cp /content/sent_simplification/sari.py /content/easse/easse

In [ ]:
%cd easse
! pip install .

In [ ]:
! mkdir /content/drive/MyDrive/SImplification_models/easse_baseline_res
! mkdir prepared_data
!mkdir preds

Prepare data for SARI

In [50]:
! python /content/sent_simplification/refs_to_easse_format.py \
--input_path /content/drive/MyDrive/SImplification_models/test_baselines.csv \
--output_dataset_name test_ref_data \
--src_column "INPUT:source" \
--trg_column "OUTPUT:output" \
--output_dir /content/prepared_data

1000
3406
3406
Overall number of references: 3406


In [46]:
with open('/content/prepared_data/gpt_bs_pred.hyp', 'w+') as f:
  for i in test_data['gpt_bs'].values:
      f.write(re.sub(r'[\t\n\r\f\v]{1,5}', "",i)+'\n')

with open('/content/prepared_data/trunctuation_bs_pred.hyp', 'w+') as f:
  for i in test_data['trunctuation_bs'].values:
      f.write(i+'\n')


In [39]:
with open('/content/prepared_data/dst_sent.txt', 'w+') as f:
  for i in test_data['OUTPUT:output'].values:
      f.write(re.sub(r'[\t\n\r\f\v]{1,10}', "",i)+'\n')


In [ ]:
! mkdir /content/preds/test
%cp /content/preds/pred_tr_bs.tok /content/preds/test/pred_tr_bs.raw # ???

In [56]:
# EXPERIMENT
# test_data['INPUT:source'] = test_data['INPUT:source'].apply(lambda x: re.sub(r'[\t\n\r\f\v]{1,10}', "",x))
# test_data['OUTPUT:output'] = test_data['OUTPUT:output'].apply(lambda x: re.sub(r'[\t\n\r\f\v]{1,10}', "",x))
# test_data.to_csv('test_data_new.csv', index=False)

## SARI
1 Trunctuation Baseline

In [ ]:
# ! wget https://github.com/dialogue-evaluation/RuSimpleSentEval/blob/main/dev_sents.csv

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/prepared_data/trunctuation_bs_pred.hyp -q

{'sari': 32.02, 'quality_estimation': {'Compression ratio': 0.602, 'Sentence splits': 0.994, 'Levenshtein similarity': 0.348, 'Exact copies': 0.0, 'Additions proportion': 0.496, 'Deletions proportion': 0.885, 'Lexical complexity score': 10.431}}


2 GPT baseline

In [ ]:
! easse evaluate \
--test_set custom \
--metrics sari \
--refs_sents_paths /content/prepared_data/test_ref_data.ref.0,/content/prepared_data/test_ref_data.ref.1,/content/prepared_data/test_ref_data.ref.2,/content/prepared_data/test_ref_data.ref.3,/content/prepared_data/test_ref_data.ref.4 \
--orig_sents_path /content/prepared_data/test_ref_data.src \
--sys_sents_path /content/prepared_data/gpt_bs_pred.hyp -q

{'sari': 32.52, 'quality_estimation': {'Compression ratio': 0.944, 'Sentence splits': 1.01, 'Levenshtein similarity': 0.371, 'Exact copies': 0.0, 'Additions proportion': 0.732, 'Deletions proportion': 0.798, 'Lexical complexity score': 10.206}}


In [ ]:
#. ,$REFERENCES_DIR/$DATASET_NAME.ref.5,$REFERENCES_DIR/$DATASET_NAME.ref.6,$REFERENCES_DIR/$DATASET_NAME.ref.7,$REFERENCES_DIR/$DATASET_NAME.ref.8,$REFERENCES_DIR/$DATASET_NAME.ref.9,$REFERENCES_DIR/$DATASET_NAME.ref.10,$REFERENCES_DIR/$DATASET_NAME.ref.11,$REFERENCES_DIR/$DATASET_NAME.ref.12,$REFERENCES_DIR/$DATASET_NAME.ref.13,$REFERENCES_DIR/$DATASET_NAME.ref.14,$REFERENCES_DIR/$DATASET_NAME.ref.15,$REFERENCES_DIR/$DATASET_NAME.ref.16,$REFERENCES_DIR/$DATASET_NAME.ref.17,$REFERENCES_DIR/$DATASET_NAME.ref.18,$REFERENCES_DIR/$DATASET_NAME.ref.19,$REFERENCES_DIR/$DATASET_NAME.ref.20

### BLEU??

In [47]:
import sacrebleu
from sacremoses import MosesDetokenizer
md = MosesDetokenizer(lang='ru')


# Open the test dataset human translation file and detokenize the references
refs = []

with open("/content/prepared_data/dst_sent.txt") as test:
    for line in test: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        refs.append(line)
    
print("Reference 1st sentence:", refs[0])

refs = [refs]  # Yes, it is a list of list(s) as required by sacreBLEU


# Open the translation file by the NMT model and detokenize the predictions
preds = []

with open("/content/prepared_data/trunctuation_bs_pred.hyp") as pred:  
    for line in pred: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        preds.append(line)

print("MTed 1st sentence:", preds[0])    


# Calculate and print the BLEU score
bleu = sacrebleu.corpus_bleu(preds, refs)
print(bleu.score)

Reference 1st sentence: 14 декабря 1944 года рабочий посёлок Ички переименован в Советский.
MTed 1st sentence: 14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок.
12.220596636706833


In [69]:
import sacrebleu
from sacremoses import MosesDetokenizer
md = MosesDetokenizer(lang='ru')


# Open the test dataset human translation file and detokenize the references
refs = []

with open("/content/prepared_data/src_sent.txt") as test:
    for line in test: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        refs.append(line)
    
print("Reference 1st sentence:", refs[0])

refs = [refs]  # Yes, it is a list of list(s) as required by sacreBLEU


# Open the translation file by the NMT model and detokenize the predictions
preds = []

with open("/content/prepared_data/trunctuation_bs_pred.hyp") as pred:  
    for line in pred: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        preds.append(line)

print("MTed 1st sentence:", preds[0])    


# Calculate and print the BLEU score
bleu = sacrebleu.corpus_bleu(preds, refs)
print(bleu.score)

Reference 1st sentence: 14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.
MTed 1st sentence: 14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок.
48.061069407227485


In [68]:
import sacrebleu
from sacremoses import MosesDetokenizer
md = MosesDetokenizer(lang='ru')


# Open the test dataset human translation file and detokenize the references
refs = []

with open("/content/prepared_data/dst_sent.txt") as test:
    for line in test: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        refs.append(line)
    
print("Reference 1st sentence:", refs[0])

refs = [refs]  # Yes, it is a list of list(s) as required by sacreBLEU


# Open the translation file by the NMT model and detokenize the predictions
preds = []

with open("/content/prepared_data/gpt_bs_pred.hyp") as pred:  
    for line in pred: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        preds.append(line)

print("MTed 1st sentence:", preds[0])    


# Calculate and print the BLEU score
bleu = sacrebleu.corpus_bleu(preds, refs)
print(bleu.score)

Reference 1st sentence: 14 декабря 1944 года рабочий посёлок Ички переименован в Советский.
MTed 1st sentence: 14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, который тогда назывался Рабоче-Крестьянской республикой.
9.86552026729868


In [49]:
import sacrebleu
from sacremoses import MosesDetokenizer
md = MosesDetokenizer(lang='ru')


# Open the test dataset human translation file and detokenize the references
refs = []

with open("/content/prepared_data/src_sent.txt") as test:
    for line in test: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        refs.append(line)
    
print("Reference 1st sentence:", refs[0])

refs = [refs]  # Yes, it is a list of list(s) as required by sacreBLEU


# Open the translation file by the NMT model and detokenize the predictions
preds = []

with open("/content/prepared_data/gpt_bs_pred.hyp") as pred:  
    for line in pred: 
        line = line.strip().split() 
        line = md.detokenize(line) 
        preds.append(line)

print("MTed 1st sentence:", preds[0])    


# Calculate and print the BLEU score
bleu = sacrebleu.corpus_bleu(preds, refs)
print(bleu.score)

Reference 1st sentence: 14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.
MTed 1st sentence: 14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, который тогда назывался Рабоче-Крестьянской республикой.
56.42809934158597


# Flesch-Kincard Grade Level

http://ceur-ws.org/Vol-2780/paper2.pdf

In [ ]:
textstat.set_lang('ru')

### Original Sentences and their simplifications scores

Flesh Kincaid Grade Level

In [ ]:
a, b = test_data['INPUT:source'].values, test_data['OUTPUT:output'].values
orig = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
simpl_ref = sum(textstat.flesch_kincaid_grade(i) for i in b)/len(b)
orig, simpl_ref

(22.591720493247266, 19.4141808573106)

Syllables number

In [ ]:
orig = sum(textstat.syllable_count(i) for i in a)/len(a)
simpl_ref = sum(textstat.syllable_count(i) for i in b)/len(b)
orig, simpl_ref

(46.53758073987082, 31.318261890780974)

Words number

In [ ]:
orig = sum(textstat.lexicon_count(i, removepunct=True) for i in a)/len(a)
simpl_ref = sum(textstat.lexicon_count(i, removepunct=True) for i in b)/len(b)
orig, simpl_ref

(17.69583088667058, 12.413681738109219)

### Trunctuation baseline

Flesh Kincaid Grade Level

In [ ]:
a, b = test_data['trunctuation_bs'].values, test_data['gpt_bs'].values
tr_bs = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
gpt_bs = sum(textstat.flesch_kincaid_grade(i) for i in b)/len(b)
tr_bs, gpt_bs

(19.601438637698003, 20.97997651203749)

Syllables number

In [ ]:
orig = sum(textstat.syllable_count(i) for i in a)/len(a)
simpl_ref = sum(textstat.syllable_count(i) for i in b)/len(b)
orig, simpl_ref

(26.924544920728128, 41.74515560775103)

Words number

In [ ]:
orig = sum(textstat.lexicon_count(i, removepunct=True) for i in a)/len(a)
simpl_ref = sum(textstat.lexicon_count(i, removepunct=True) for i in b)/len(b)
orig, simpl_ref

(10.268937169700529, 16.581620669406927)

### GPT baseline

Flesh Kincaid Grade Level

In [ ]:
a, b = test_data['gpt_bs'].values, test_data['gpt_bs'].values
tr_bs = sum(textstat.flesch_kincaid_grade(i) for i in a)/len(a)
gpt_bs = sum(textstat.flesch_kincaid_grade(i) for i in b)/len(b)
tr_bs, gpt_bs

Syllables number

In [ ]:
orig = sum(textstat.syllable_count(i) for i in a)/len(a)
simpl_ref = sum(textstat.syllable_count(i) for i in b)/len(b)
orig, simpl_ref

Words number

In [ ]:
orig = sum(textstat.lexicon_count(i, removepunct=True) for i in a)/len(a)
simpl_ref = sum(textstat.lexicon_count(i, removepunct=True) for i in b)/len(b)
orig, simpl_ref

In [ ]:
# ???????
# >>> textstat.flesch_reading_ease(test_data)
# >>> textstat.smog_index(test_data)
# >>> textstat.flesch_kincaid_grade(test_data)
# >>> textstat.coleman_liau_index(test_data)
# >>> textstat.automated_readability_index(test_data)
# >>> textstat.dale_chall_readability_score(test_data)
# >>> textstat.difficult_words(test_data)
# >>> textstat.linsear_write_formula(test_data)
# >>> textstat.gunning_fog(test_data)
# >>> textstat.text_standard(test_data)
# >>> textstat.fernandez_huerta(test_data)
# >>> textstat.szigriszt_pazos(test_data)
# >>> textstat.gutierrez_polini(test_data)
# >>> textstat.crawford(test_data)

## Grammar

In [ ]:
! gdown https://drive.google.com/uc?id=1IVz3XC8Rm7hQCyx3xCcABjhaENrKABvF
test_data = pd.read_csv('/content/test_baselines.csv', sep='\t')

In [7]:
tool = language_tool_python.LanguageTool('ru')

Unzipping /tmp/tmpds0_hg1n.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.2.zip to /root/.cache/language_tool_python.


In [8]:
def get_mistakes_summary(df_test, x, y):
    src_test = list(df_test[x].values)
    dst_test =list(df_test[y].values)
    matches_src = []
    for i in src_test:
      matches_src.extend(tool.check(i))
    matches_src

    matches_dst = []
    for i in dst_test:
      matches_dst.extend(tool.check(i))
    matches_dst

    categories = set([i.category for i in matches_src+matches_dst])

    categories_src = {i:0 for i in categories}
    categories_dst = {i:0 for i in categories}

    for i in matches_src:
      categories_src[i.category]+=1

    for i in matches_dst:
      categories_dst[i.category]+=1
      
    return categories_src, categories_dst

In [10]:
test_data.columns

Index(['Unnamed: 0', 'INPUT:source', 'OUTPUT:output', 'trunctuation_bs', 'gpt_bs', 'cos_sim_ref', 'cos_sim_tr_bs', 'cos_sim_gpt_bs'], dtype='object')

In [11]:
src_errors, dst_errors = get_mistakes_summary(test_data, 'INPUT:source', 'OUTPUT:output')

In [12]:
src_errors

{'CASING': 10,
 'EXTEND': 0,
 'GRAMMAR': 22,
 'LOGIC': 27,
 'MISC': 0,
 'PUNCTUATION': 34,
 'STYLE': 14,
 'TYPOGRAPHY': 33,
 'TYPOS': 809}

In [13]:
dst_errors

{'CASING': 144,
 'EXTEND': 1,
 'GRAMMAR': 110,
 'LOGIC': 15,
 'MISC': 28,
 'PUNCTUATION': 106,
 'STYLE': 7,
 'TYPOGRAPHY': 733,
 'TYPOS': 746}

In [14]:
src_errors, dst_errors = get_mistakes_summary(test_data, 'trunctuation_bs', 'gpt_bs')

In [15]:
src_errors

{'CASING': 0,
 'GRAMMAR': 50,
 'LOGIC': 22,
 'MISC': 0,
 'PUNCTUATION': 43,
 'STYLE': 203,
 'TYPOGRAPHY': 487,
 'TYPOS': 441}

In [16]:
dst_errors

{'CASING': 2,
 'GRAMMAR': 46,
 'LOGIC': 23,
 'MISC': 3,
 'PUNCTUATION': 54,
 'STYLE': 4,
 'TYPOGRAPHY': 76,
 'TYPOS': 697}